In [ ]:
import pandas

import automol
import chemkin_io
import helpers
import ioformat

In [ ]:
# 1. read in the species and reaction lists
FUEX_C67_SPC_PATH = 'species/02_species-fu-exp_nuig-c6-7.csv'
ORIG_C67_RXN_PATH = 'reactions/03_reactions-succ_nuig-c6-7.csv'

FUEX_PYR_SPC_PATH = 'species/02_species-fu-exp_nuig-pyro.csv'
ORIG_PYR_RXN_PATH = 'reactions/03_reactions-succ_nuig-pyro.csv'

FUEX_C67_SPC_DF = pandas.read_csv(FUEX_C67_SPC_PATH, quotechar="'")
ORIG_C67_RXN_DF = pandas.read_csv(ORIG_C67_RXN_PATH, quotechar="'")
FUEX_PYR_SPC_DF = pandas.read_csv(FUEX_PYR_SPC_PATH, quotechar="'")
ORIG_PYR_RXN_DF = pandas.read_csv(ORIG_PYR_RXN_PATH, quotechar="'")

In [ ]:
# 2. expand reaction stereo (full and non-redundant) for each list
def expand_reaction_stereo(rxn_df, spc_df, enant=True):
    """ expand stereo for a species DataFrame

        :param rxn_df: A reaction DataFrame containing
            a 'name' column with CHEMKIN chemical
            equations and a 'path' column with paths to
            serialized reaction objects.
        :param spc_df: the species DataFrame, which must contain the
            following columns: 'inchi', 'orig-name', 'name', and 'smiles'
        :param enant: If True, perform a full expansion
            including all enantiomers. If False, perform a
            non-redundant expansion with only canonical
            enantiomers.
    """
    row_dcts = []
    count = len(rxn_df.index)
    for num, row in rxn_df.iterrows():
        print(f'Row {num}/{count}')
        name = row['name']
        print(f'Expanding {name}...')
        class_ = row['class']
        print(f' - Class: {class_}')
        rct_names, prd_names, third_body = chemkin_io.parser.reaction.get_rxn_name(name)
        rxn_path = row['path']
        print(f' - Reading from {rxn_path}')
        rxn_obj_str = ioformat.pathtools.read_file('.', rxn_path)
        rxn_obj = automol.reac.from_string(rxn_obj_str)
        ste_rxn_objs = automol.reac.expand_stereo(rxn_obj, enant=enant)
        for idx, ste_rxn_obj in enumerate(ste_rxn_objs):
            row_dct = row.to_dict()
            # get ChIs for reactants and products
            ste_rct_chis, ste_prd_chis = automol.reac.chis(ste_rxn_obj)
            # get the ChI for the TS
            ste_ts_chi = automol.reac.ts_amchi(ste_rxn_obj)
            # format the CHEMKIN chemical equation string
            ste_rct_names = [
                helpers.lookup_value(spc_df, 'name', {'inchi': c, 'orig-name': n})
                for c, n in zip(ste_rct_chis, rct_names)]
            ste_prd_names = [
                helpers.lookup_value(spc_df, 'name', {'inchi': c, 'orig-name': n})
                for c, n in zip(ste_prd_chis, prd_names)]
            ste_name = chemkin_io.writer.format_rxn_name(
                (ste_rct_names, ste_prd_names, third_body))
            print(f' {idx+1:02d}. {ste_name}')
            # write the reaction object with stereo to file
            ste_rxn_obj_str = automol.reac.string(ste_rxn_obj)
            ste_rxn_obj_path = row_dct['path'].replace('.txt', f'_{"fu" if enant else "nr"}{idx}.txt')
            ioformat.pathtools.write_file(ste_rxn_obj_str, '.', ste_rxn_obj_path)
            print(f'   - Writing to {ste_rxn_obj_path}')
            row_dct.update({
                'name': ste_name,
                'path': ste_rxn_obj_path,
                'ts_amchi_stereo': ste_ts_chi,
            })
            row_dcts.append(row_dct)
            # show SMILES for the reaction
            ste_rct_smis = [
                helpers.lookup_value(spc_df, 'smiles', {'inchi': c, 'orig-name': n})
                for c, n in zip(ste_rct_chis, rct_names)]
            ste_prd_smis = [
                helpers.lookup_value(spc_df, 'smiles', {'inchi': c, 'orig-name': n})
                for c, n in zip(ste_prd_chis, prd_names)]
            ste_rxn_smi = automol.smiles.reaction(ste_rct_smis, ste_prd_smis)
            print(f'   - SMILES: {ste_rxn_smi}')
            # display the reaction
            automol.chi.display_reaction(ste_rct_chis, ste_prd_chis)
    ste_rxn_df = pandas.DataFrame.from_records(row_dcts)
    return ste_rxn_df

In [ ]:
print("Full reaction stereoexpansion for NUIG-C6-7")
print("WARNING: RDKit fails to properly visualize some radicals")
FUEX_C67_RXN_DF = expand_reaction_stereo(ORIG_C67_RXN_DF, FUEX_C67_SPC_DF, enant=True)

FUEX_C67_RXN_PATH = 'reactions/04_reactions-fu-exp_nuig-c6-7.csv'
FUEX_C67_RXN_DF.to_csv(FUEX_C67_RXN_PATH, quotechar="'")

In [ ]:
print("Non-redundant expansion for NUIG-C6-7")
print("WARNING: RDKit fails to properly visualize some radicals")
NREX_C67_RXN_DF = expand_reaction_stereo(ORIG_C67_RXN_DF, FUEX_C67_SPC_DF, enant=False)

NREX_C67_RXN_PATH = 'reactions/04_reactions-nr-exp_nuig-c6-7.csv'
NREX_C67_RXN_DF.to_csv(NREX_C67_RXN_PATH, quotechar="'")

In [ ]:
print("Full expansion for NUIG-Pyro")
print("WARNING: RDKit fails to properly visualize some radicals")
FUEX_PYR_RXN_DF = expand_reaction_stereo(ORIG_PYR_RXN_DF, FUEX_PYR_SPC_DF, enant=True)

FUEX_PYR_RXN_PATH = 'reactions/04_reactions-fu-exp_nuig-pyro.csv'
FUEX_PYR_RXN_DF.to_csv(FUEX_PYR_RXN_PATH, quotechar="'")

In [ ]:
print("Non-redundant expansion for NUIG-Pyro")
print("WARNING: RDKit fails to properly visualize some radicals")
NREX_PYR_RXN_DF = expand_reaction_stereo(ORIG_PYR_RXN_DF, FUEX_PYR_SPC_DF, enant=False)

NREX_PYR_RXN_PATH = 'reactions/04_reactions-nr-exp_nuig-pyro.csv'
NREX_PYR_RXN_DF.to_csv(NREX_PYR_RXN_PATH, quotechar="'")